In [7]:
import pandas as pd

matches_df = pd.read_csv("datasets/WorldCupMatches.csv")
players_df = pd.read_csv("datasets/WorldCupPlayers.csv")
worldcups_df = pd.read_csv("datasets/WorldCups.csv")

print("=== WorldCupMatches Columns ===")
print(matches_df.columns)

print("=== WorldCupPlayers Columns ===")
print(players_df.columns)

print("=== WorldCups Columns ===")
print(worldcups_df.columns)

=== WorldCupMatches Columns ===
Index(['Year', 'Datetime', 'Stage', 'Stadium', 'City', 'Home Team Name',
       'Home Team Goals', 'Away Team Goals', 'Away Team Name',
       'Win conditions', 'Attendance', 'Half-time Home Goals',
       'Half-time Away Goals', 'Referee', 'Assistant 1', 'Assistant 2',
       'RoundID', 'MatchID', 'Home Team Initials', 'Away Team Initials'],
      dtype='object')
=== WorldCupPlayers Columns ===
Index(['RoundID', 'MatchID', 'Team Initials', 'Coach Name', 'Line-up',
       'Shirt Number', 'Player Name', 'Position', 'Event'],
      dtype='object')
=== WorldCups Columns ===
Index(['Year', 'Country', 'Winner', 'Runners-Up', 'Third', 'Fourth',
       'GoalsScored', 'QualifiedTeams', 'MatchesPlayed', 'Attendance'],
      dtype='object')


# Data Pre-Processing

In [8]:
matches_df.dropna(how='all', inplace=True)
matches_df = matches_df.dropna(subset=['Year'])
matches_df['Year'] = matches_df['Year'].astype(int)
matches_df['Datetime'] = pd.to_datetime(matches_df['Datetime'], errors='coerce')

goal_cols = ['Home Team Goals', 'Away Team Goals', 'Half-time Home Goals', 'Half-time Away Goals']
matches_df[goal_cols] = matches_df[goal_cols].astype('Int64')

matches_df['Attendance'] = pd.to_numeric(matches_df['Attendance'], errors='coerce').round().astype('Int64')

matches_df['Home Team Name'] = matches_df['Home Team Name'].str.strip()
matches_df['Away Team Name'] = matches_df['Away Team Name'].str.strip()

players_df['Shirt Number'] = pd.to_numeric(players_df['Shirt Number'], errors='coerce')

worldcups_df['Attendance'] = worldcups_df['Attendance'].replace('[\.,]', '', regex=True).astype(int)
worldcups_df['Year'] = worldcups_df['Year'].astype(int)

# Feature Engineering

### Creating a new match winner column

In [10]:
def get_match_winner(row):
    if row['Home Team Goals'] > row['Away Team Goals']:
        return 'Home'
    elif row['Home Team Goals'] < row['Away Team Goals']:
        return 'Away'
    else:
        return 'Draw'

matches_df['match_winner'] = matches_df.apply(get_match_winner, axis=1)
matches_df['goal_difference'] = (matches_df['Home Team Goals'] - matches_df['Away Team Goals']).abs()
matches_df['total_goals'] = matches_df['Home Team Goals'] + matches_df['Away Team Goals']

print(matches_df[['Year', 'Home Team Name', 'Away Team Name', 'Home Team Goals', 'Away Team Goals', 'match_winner', 'goal_difference', 'total_goals']].head())

   Year Home Team Name Away Team Name  Home Team Goals  Away Team Goals  \
0  1930         France         Mexico                4                1   
1  1930            USA        Belgium                3                0   
2  1930     Yugoslavia         Brazil                2                1   
3  1930        Romania           Peru                3                1   
4  1930      Argentina         France                1                0   

  match_winner  goal_difference  total_goals  
0         Home                3            5  
1         Home                3            3  
2         Home                1            3  
3         Home                2            4  
4         Home                1            1  
